In [8]:
import numpy as np
import scipy as sp
import matplotlib as mlp
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [11]:
plt.style.use('dark_background')

In [14]:
plt.rcParams['figure.figsize'] = (10,5)
plt.rcParams['figure.dpi'] = 150

# Interaction Basics: `interact` and `@interact`
This notebook is inspired by https://www.youtube.com/watch?v=rkBPgTL-D3Y     
The doc is great: https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html           
The `interact` function automatically choose the appropriate widget for interaction according to the variable type.
## Real number variables

In [15]:
def f(x):
    plt.plot( np.arange(0,10), x*np.arange(0,10) )
    plt.ylim(-30,30)

In [42]:
interact(f, x=1) #Simple interaction starting at x = 1

interactive(children=(IntSlider(value=1, description='x', max=3, min=-1), Output()), _dom_classes=('widget-int…

<function __main__.f(x)>

In [43]:
interact(f, x=(-3,3,0.05)) #Simple interaction with min,max,step information on the variable

interactive(children=(FloatSlider(value=0.0, description='x', max=3.0, min=-3.0, step=0.05), Output()), _dom_c…

<function __main__.f(x)>

## Boolean variables

In [44]:
rand = np.random.rand(100)
def g(x):
    if x:
        plt.plot( rand, 'b' )
    else:
        plt.plot( rand, 'r' )

In [45]:
interact(g, x=True)

interactive(children=(Checkbox(value=True, description='x'), Output()), _dom_classes=('widget-interact',))

<function __main__.g(x)>

## String variables
Another syntax to use interact is through decoration.

In [46]:
@interact( x = "Hello" )
def h(x):
    plt.title( x )
    plt.plot(rand, rand)

interactive(children=(Text(value='Hello', description='x'), Output()), _dom_classes=('widget-interact',))

## Multiple variables

The module interprets tuples as sliders but we can define the slider ourself. Don't share this slider object, i.e. dont use it for both x and y.

In [47]:
w = widgets.FloatSlider( value = 1, min = -10, max = 10, step = 0.05 )
@interact( x = w, y = (-10,10,0.05), title = "My nice stuff")
def i(x,y,title):
    plt.title(title)
    plt.plot(np.arange(10), x*np.arange(10)+y)
    

interactive(children=(FloatSlider(value=1.0, description='x', max=10.0, min=-10.0, step=0.05), FloatSlider(val…

## Fixing values

In [34]:
interact(i, x = w, y = fixed( 3 ), title = "My nice stuff")

interactive(children=(FloatSlider(value=1.0, description='x', max=10.0, min=-10.0, step=0.05), Text(value='My …

<function __main__.i(x, y, title)>

## Dropdowns

In [38]:
@interact( x = w, y = (-10,10,0.05), color = ['blue', 'red', 'yellow'])
def j(x,y,color):
    plt.plot(np.arange(10), x*np.arange(10)+y, c = color)

interactive(children=(FloatSlider(value=1.0, description='x', max=10.0, min=-10.0, step=0.05), FloatSlider(val…

In [41]:
@interact( x = w, y = {'one': 1, 'two': 2})
def h(x,y):
    plt.plot(np.arange(10), x*np.arange(10)+y)

interactive(children=(FloatSlider(value=1.0, description='x', max=10.0, min=-10.0, step=0.05), Dropdown(descri…

## Slow function: `interact_manual`.
Continuous update might be really annoying when the function behind is slow.    
The doc outlines several ways to deal with that issue. My preferred one is to use `interact_manual`.

In [48]:
def slow_function(i):
    print(int(i),list(x for x in range(int(i)) if
                str(x)==str(x)[::-1] and
                str(x**2)==str(x**2)[::-1]))
    return

In [49]:
%%time
slow_function(1e6)

1000000 [0, 1, 2, 3, 11, 22, 101, 111, 121, 202, 212, 1001, 1111, 2002, 10001, 10101, 10201, 11011, 11111, 11211, 20002, 20102, 100001, 101101, 110011, 111111, 200002]
CPU times: user 562 ms, sys: 981 µs, total: 563 ms
Wall time: 561 ms


In [53]:
interact_manual(slow_function,i=widgets.FloatSlider(min=1e5, max=1e7, step=1e5));

interactive(children=(FloatSlider(value=100000.0, description='i', max=10000000.0, min=100000.0, step=100000.0…

# More liberty: `interactive` and `interactive_output`
## `interactive`     
`interact` is nice but does not give an handle on the function result. You can also embed the interactive function in a more wider context.

In [57]:
from IPython.display import display

In [135]:
def func(a, b):
    display(a + b)
    return a+b

In [164]:
w = interactive(func,  a=10, b=20)

In [165]:
w = interactive(func,  **{'a':10, 'b':20})

In [153]:
type(w)

ipywidgets.widgets.interaction.interactive

In [159]:
display(widgets.VBox([widgets.Label('Wider context'),w]))

In [161]:
w.children

(IntSlider(value=9, description='a', max=30, min=-10),
 IntSlider(value=20, description='b', max=60, min=-20),
 Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '29'}, 'metadata': {}},)))

**Note that:** All the widgets are available in `w.children` thus it is easy to integrate in whatever way in a custom UI.   
For instance plotting the original UI upside down.

In [160]:
display(widgets.VBox([widgets.Label('Wider context'),*w.children[::-1]]))

In [142]:
print( w.kwargs, w.result ) 

{'a': 10, 'b': 20} 30


## Minixing `interactive` with `interact_manual`

In [150]:
w = interactive(func, {'manual': True, "auto_display":False}, a=10, b=20)
display(widgets.VBox([widgets.Label('Wider context'),w]))

## `interactive_output`

 `interactive_output` gives a handle only on the interactive output, it doenst construct the UI whatsoever. It is interesting to integrate a interactive output (i.e. a plot) in a complex gui.       
 But I feel like everything can be done using only `interactive` (you have the output in `interactive(func,  a=10, b=20).children[-1]`).


In [149]:
a = widgets.FloatSlider( value = 1, min = -10, max = 10, step = 0.05, description = 'a' )
b = widgets.FloatSlider( value = 1, min = -10, max = 10, step = 0.05, description = 'b' )
c = widgets.FloatSlider( value = 1, min = -10, max = 10, step = 0.05, description = 'c' )
ui = widgets.VBox([a, b, c])
def foo(a, b, c):
    plt.plot(np.arange(10), a*np.arange(10)+b+c)
    return a + b + c

out = widgets.interactive_output(foo, {'a': a, 'b': b, 'c': c})

display(widgets.VBox([ out, ui ] ) )